# Demo: Evaluating Normality and Distributional Differences in Synthetic DatasetsEvaluating Normality and Distributional Differences in Synthetic Datasets.

In [ ]:
%pip install numpy pandas matplotlib 
%pip install scipy.stats

In [29]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import (kstest, ks_2samp, shapiro, ttest_ind, chi2_contingency,
                         gaussian_kde, anderson, t)
from matplotlib.backends.backend_pdf import PdfPages
import io
import sys

# Set random seed for reproducibility
np.random.seed(42)

# Number of samples
n = 1000000
mean_A = 50
std_A = 10
data_A = np.random.normal(loc=mean_A, scale=std_A, size=n)

# Non-normal distribution (Gamma)
shape_B = 5.0
scale_B = 10.0
data_B = np.random.gamma(shape=shape_B, scale=scale_B, size=n)

# Choose subset size for normality test
subset_size = 5000

# Random subsets of data for normality test
subset_indices_A = np.random.choice(len(data_A), subset_size, replace=False)
subset_indices_B = np.random.choice(len(data_B), subset_size, replace=False)

data_A_subset = data_A[subset_indices_A]
data_B_subset = data_B[subset_indices_B]

alpha = 0.05

# Capture all print statements
buffer = io.StringIO()
old_stdout = sys.stdout
sys.stdout = buffer  # Redirect print statements to buffer

# Shapiro-Wilk Test on subsets because Shapiro-Wilk
# gives inflated p values for large datasets
stat_A, p_A = shapiro(data_A_subset)
stat_B, p_B = shapiro(data_B_subset)

print("Shapiro-Wilk Test for Normality (done on subsets because N>5000):")
print(f"Dataset A subset: statistic = {stat_A:.4f}, p-value = {p_A:.4f}")
print(f"Dataset B subset: statistic = {stat_B:.4f}, p-value = {p_B:.4f}")

shapiro_normal_A = (p_A > alpha)
shapiro_normal_B = (p_B > alpha)

# Anderson-Darling Test
ad_result_A = anderson(data_A, dist='norm')
ad_result_B = anderson(data_B, dist='norm')

print("\nAnderson-Darling Test for Normality (on full datasets):")
print(f"Dataset A subset: A2 = {ad_result_A.statistic:.4f}")
for i, cv in enumerate(ad_result_A.critical_values):
    sl = ad_result_A.significance_level[i]
    print(f"  Significance level: {sl:.1f}%, Critical Value: {cv:.4f}")
print(f"Dataset B subset: A2 = {ad_result_B.statistic:.4f}")
for i, cv in enumerate(ad_result_B.critical_values):
    sl = ad_result_B.significance_level[i]
    print(f"  Significance level: {sl:.1f}%, Critical Value: {cv:.4f}")

# Check against the 5% significance level (usually the last one)
ad_normal_A = ad_result_A.statistic < ad_result_A.critical_values[-1]
ad_normal_B = ad_result_B.statistic < ad_result_B.critical_values[-1]

both_normal = shapiro_normal_A and shapiro_normal_B and ad_normal_A and ad_normal_B

test_summary = ""  # We'll accumulate a summary of test results to annotate

if both_normal:
    print("\nBoth subsets appear normally distributed. Using t-test on full data...")
    mean_diff = np.mean(data_B) - np.mean(data_A)
    print("Mean(A):", np.mean(data_A))
    print("Mean(B):", np.mean(data_B))
    print("Mean Difference (B - A):", mean_diff)

    t_stat, p_val = ttest_ind(data_B, data_A)
    print("T-Test Results:")
    print("T-statistic:", t_stat)
    print("P-value:", p_val)

    # Compute 95% CI for difference in means
    # Assuming equal variances and equal sample sizes:
    var_A = np.var(data_A, ddof=1)
    var_B = np.var(data_B, ddof=1)
    se = np.sqrt((var_A/n) + (var_B/n))
    dof = 2*n - 2
    t_crit = t.ppf(1 - alpha/2, dof)
    ci_low = mean_diff - t_crit * se
    ci_high = mean_diff + t_crit * se

    test_summary += f"T-test: p-value={p_val:.4e}\n"
    test_summary += f"95% CI diff in means: [{ci_low:.2f}, {ci_high:.2f}]\n"

else:
    print("\nData is not normally distributed. Using Pearson Chi-square test on binned data...")

    # Combine data to define bins
    combined_data = np.concatenate([data_A, data_B])
    num_bins = 20
    bins = np.linspace(min(combined_data), max(combined_data), num_bins + 1)

    freq_A, _ = np.histogram(data_A, bins=bins)
    freq_B, _ = np.histogram(data_B, bins=bins)

    # Remove empty bins
    non_empty_bins = (freq_A + freq_B) > 0
    freq_A = freq_A[non_empty_bins]
    freq_B = freq_B[non_empty_bins]

    contingency_table = np.vstack([freq_A, freq_B])

    chi2_stat, chi2_p, dof, expected = chi2_contingency(contingency_table)
    print("Chi-square Test Results:")
    print("Chi-square statistic:", chi2_stat)
    print("Degrees of freedom:", dof)
    print("P-value:", chi2_p)

    test_summary += f"Chi-square test: p-value={chi2_p:.4e}\n"

# KS Test (For Distributional Differences)
ks_stat, ks_p_val = ks_2samp(data_A, data_B)
print("\nKS Test Results (For Distributional Differences):")
print("KS-statistic:", ks_stat)
print("P-value:", ks_p_val)
test_summary += f"KS-test: p-value={ks_p_val:.4e}"

# Restore original stdout
sys.stdout = old_stdout

# Get all printed output as a string
text_output = buffer.getvalue()
buffer.close()


In [30]:
# ---- Plotting (on full datasets) ----
fig1, axes = plt.subplots(2, 2, figsize=(12, 8))

# We'll use red for Dataset A and blue for Dataset B
color_A = 'red'
color_B = 'blue'

# Histograms
axes[0, 0].hist(data_A, bins=30, alpha=0.7, color=color_A, label='Dataset A')
axes[0, 0].hist(data_B, bins=30, alpha=0.7, color=color_B, label='Dataset B')
axes[0, 0].axvline(np.mean(data_A), color=color_A, linestyle='dashed', linewidth=2)
axes[0, 0].axvline(np.mean(data_B), color=color_B, linestyle='dashed', linewidth=2)
axes[0, 0].set_title("Histogram of Data")
axes[0, 0].set_xlabel("Value")
axes[0, 0].set_ylabel("Frequency")
axes[0, 0].legend()

# Boxplots
bplot = axes[0, 1].boxplot([data_A, data_B], tick_labels=['Dataset A', 'Dataset B'], patch_artist=True,
                           boxprops=dict(facecolor=color_A), medianprops=dict(color='black'))
bplot['boxes'][1].set_facecolor(color_B)
bplot['whiskers'][2].set_color(color_B)
bplot['whiskers'][3].set_color(color_B)
bplot['caps'][2].set_color(color_B)
bplot['caps'][3].set_color(color_B)
bplot['medians'][1].set_color('black')
axes[0, 1].set_title("Boxplot Comparison")

# Add annotation with p-values and/or CI above the boxplot
axes[0, 1].text(0.5, 0.8, test_summary, ha='center', va='top', transform=axes[0,1].transAxes,
                bbox=dict(facecolor='white', alpha=0.8))

# CDF plots
sorted_A = np.sort(data_A)
sorted_B = np.sort(data_B)
cdf_A = np.arange(1, n+1) / n
cdf_B = np.arange(1, n+1) / n

axes[1, 0].plot(sorted_A, cdf_A, label='Dataset A CDF', linewidth=2, color=color_A)
axes[1, 0].plot(sorted_B, cdf_B, label='Dataset B CDF', linewidth=2, color=color_B)
axes[1, 0].set_title("CDF of Datasets")
axes[1, 0].set_xlabel("Value")
axes[1, 0].set_ylabel("Cumulative Probability")
axes[1, 0].legend()

# Annotate KS test p-value on the CDF plot
axes[1, 0].text(0.5, 0.1, f"KS p-value: {ks_p_val:.4e}", ha='center', va='bottom',
                transform=axes[1,0].transAxes, bbox=dict(facecolor='white', alpha=0.8))

# Density (KDE) Comparison
kde_A = gaussian_kde(data_A)
kde_B = gaussian_kde(data_B)
x_vals = np.linspace(min(data_A.min(), data_B.min()) - 10,
                     max(data_A.max(), data_B.max()) + 10, 200)
axes[1, 1].plot(x_vals, kde_A(x_vals), label='Dataset A KDE', linewidth=2, color=color_A)
axes[1, 1].plot(x_vals, kde_B(x_vals), label='Dataset B KDE', linewidth=2, color=color_B)
axes[1, 1].set_title("Density (KDE) Comparison")
axes[1, 1].set_xlabel("Value")
axes[1, 1].set_ylabel("Density")
axes[1, 1].legend()

plt.tight_layout()

# Create a figure for text output
fig2 = plt.figure(figsize=(9, 12))
plt.axis('off')
plt.text(0.01, 0.99, text_output, fontsize=10, va='top')

# Save all figures into a single PDF
with PdfPages('./reports/ci_report.pdf') as pdf:
    pdf.savefig(fig2)   # Text output as the first page
    pdf.savefig(fig1)   # The figure with histograms, boxplots, CDF, KDE

plt.close(fig1)
plt.close(fig2)

print("Report saved as 'ci_report.pdf'")

Report saved as 'ci_report.pdf'
